In [ ]:
!pip install selenium
!pip install python-slugify
!apt-get update
!apt install chromium-chromedriver


In [3]:
from selenium import webdriver
from slugify import slugify
import pandas as pd

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver', chrome_options=chrome_options)

In [5]:
wd.get("https://www.thespruce.com/plants-a-to-z-5116344")
cards = wd.find_elements_by_class_name('alphabetical-card')

In [6]:
links = []
for card in cards:
  links.append(card.get_attribute('href'))
del links[:8]

In [7]:
def split_list(array, elem, type = 1):
  for idx, el in enumerate(array):
    if el == elem:
      return array[:idx] if type == 1 else array[idx+1:]

In [20]:
def convert_dict(array):  
    init = iter(array)
    res_dct = dict(zip(init, init))  
    return res_dct

In [23]:
def inspect_plant(link):
  wd.get(link)
  link = wd.find_element_by_xpath('/html/body/main/article/div[1]/div/div[2]/div[2]')

  page_list = link.text.split('\n')
  if 'FEATURED VIDEO' in page_list: page_list.remove('FEATURED VIDEO')

  table = link.find_element_by_tag_name('table')

  # Get description
  description = split_list(page_list, table.text.split('\n')[0])
  description = ''.join(description)

  # Get table
  table = table.find_elements_by_tag_name('td')
  dict_table = []
  for (idx, column) in enumerate(table):
    text = column.text
    if idx % 2 == 0:
      text = slugify(text)
    dict_table.append(text)
  dict_table = convert_dict(dict_table)

  return {'description': description, **dict_table}

In [26]:
res = []
for link in links[:3]:
  res.append(inspect_plant(link))

df = pd.DataFrame(res)
df